## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

Cкачиваем пакеты.

In [ ]:
%%capture
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import  precision_score, recall_score, f1_score, accuracy_score

In [ ]:
import string
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/sismetanin/rureviews/master/women-clothing-accessories.3-class.balanced.csv'
data = pd.read_csv(url, sep='\t', usecols=[0, 1])
display(data.head())

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


Убираем знаки препинания, приводим к нижнему регистру.

In [ ]:
def remove_punct(sentence):
  for sym in string.punctuation:
    sentence = sentence.replace(sym,"")
  return sentence
data1 = data
data1['review'] = data.apply(lambda row: remove_punct(row['review']), axis=1)
data2 = data1
data2['review'] = data1.apply(lambda row: row['review'].lower(), axis=1)


Токенизируем

In [ ]:
from nltk.tokenize import word_tokenize
data3 = data2
data3['review'] = data2.apply(lambda row: word_tokenize(row['review'], language='russian'), axis=1)

Удаление стоп слов. Я решил оставить в стоп словах слово не т.к. мне кажется такие сочетания как "не нравится", "не красивый" будут информативны для анализа. Нашел даже статью на эту тему но в подробности глубоко не лез.
https://www.cs.cmu.edu/~ark/EMNLP-2015/proceedings/WASSA/pdf/WASSA14.pdf

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.remove("не")
def remove_stop_words(sentence):
  return [w for w in sentence if not w in stop_words]
data4 = data3
data4['review'] = data3.apply(lambda row: remove_stop_words(row['review']), axis=1)
display(data.head())

,review,sentiment
0,"[качество, плохое, пошив, ужасный, горловина, ...",negative
1,"[товар, отдали, другому, человеку, не, получил...",negative
2,"[ужасная, синтетика, тонкая, общего, представл...",negative
3,"[товар, не, пришел, продавец, продлил, защиту,...",negative
4,"[кофточка, голая, синтетика, носить, не, возмо...",negative


Лемматизация. Удаляем стоп слова снова чтобы векторайзер лучше работал

In [ ]:
Morph = MorphAnalyzer()
def lemmatize(sentence):
  return [Morph.parse(w)[0].normal_form for w in sentence]

data5 = data4
data5['review'] = data4.apply(lambda row: lemmatize(row['review']), axis=1)
data6 = data5
data6['review'] = data5.apply(lambda row: remove_stop_words(row['review']), axis=1)
display(data.head())

,review,sentiment
0,"[качество, плохой, пошив, ужасный, горловина, ...",negative
1,"[товар, отдать, человек, не, получить, посылка...",negative
2,"[ужасный, синтетик, тонкий, общий, представить...",negative
3,"[товар, не, прийти, продавец, продлить, защита...",negative
4,"[кофточка, голый, синтетик, носить, не, возможно]",negative


In [ ]:
def convert_to_string(sentence):
  return ' '.join(sentence)
data7 = data6
data7['review'] = data6.apply(lambda row: convert_to_string(row['review']), axis=1)
display(data.head())

,review,sentiment
0,качество плохой пошив ужасный горловина напере...,negative
1,товар отдать человек не получить посылка ладны...,negative
2,ужасный синтетик тонкий общий представить карт...,negative
3,товар не прийти продавец продлить защита согла...,negative
4,кофточка голый синтетик носить не возможно,negative


Начинаем векторизацию


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data7.review, data7.sentiment, train_size = 0.7)

Экспериментальным путем было уставлено, что лучшим параметром здесь является 1,2. Любопытно то, что до того как "не" перестало быть стоп-словом лучшим было 1,1

In [ ]:
model = Pipeline([('cv', CountVectorizer(ngram_range = (1,2))), ('classifier',MultinomialNB())])
model.fit(x_train, y_train)
count_vectorizer_y_predicted = model.predict(x_test)
cur_accuracy = accuracy_score(count_vectorizer_y_predicted, y_test)
print(cur_accuracy)

0.721751046257546


После использования gridsearch c новыми параметрами было установлено, что точность увеличивается при следующих параметрах: ngram_range = (1,2), min_df=0,
                  max_df=0.8, max_features=None

In [ ]:
model = Pipeline([('tf_idf', TfidfVectorizer(ngram_range = (1,2), min_df=0,
                  max_df=0.8, max_features=None)),('classifier',MultinomialNB())])
model.fit(x_train, y_train)
tf_idf_y_predicted = model.predict(x_test)


In [ ]:
print(accuracy_score(tf_idf_y_predicted, y_test))

0.7233065442020666


Диапазон значения для nграмм взял от 2 до 16. Почему 2? было интересно как сочетание "не" влияет на текст. Почему до 16? Потому что мне показалось что это среднее значение символов в би-триграмме

In [ ]:
model = Pipeline([('char_ngrams', CountVectorizer(analyzer='char', ngram_range = (2, 16 ))),
                  ('classifier',MultinomialNB())])
model.fit(x_train, y_train)
char_ngrams_y_predicted = model.predict(x_test)
print(accuracy_score(char_ngrams_y_predicted, y_test))

0.7267508610792193


In [ ]:
def score_report(predicted, y_test):
  presicion = precision_score(predicted, y_test, average='macro')
  recall = recall_score(predicted, y_test, average='macro')
  f1 = f1_score(predicted, y_test, average='macro')
  accuracy = accuracy_score(predicted, y_test)
  return pd.Series([presicion, recall, f1, accuracy])
 
report = pd.DataFrame({'Count vectorizer':score_report(count_vectorizer_y_predicted, y_test), 
                        'TF-IDF Vectorizer':score_report(tf_idf_y_predicted, y_test), 
                        'Char Vectorizer':score_report(char_ngrams_y_predicted, y_test)})
report.index = ['presicion','recall','f1-score', 'accuracy']
display(report.T)

,presicion,recall,f1-score,accuracy
Count vectorizer,0.721608,0.725575,0.723184,0.721751
TF-IDF Vectorizer,0.723156,0.727831,0.724888,0.723307
Char Vectorizer,0.726613,0.729519,0.727803,0.726751


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
string = "some string with a lot of words"
string = re.split(' ', string)
result = [re.findall('^..', word) for word in string]
print(result)

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
sentence = "The book studied the making and dropping of the atomic bomb from the viewpoints of the atomic scientists. The book is largely based. on personal interviews. with persons. who played. leading parts in the construction and deployment of the bombs."

result = re.split('\.',sentence,maxsplit =3)
print(result)

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result =  re.sub('\d','DIG','45 ja78 minotaur')
print(result)

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
string = "http:\\mylink.com bla bla bla"
print(re.sub(r'http\S+', '', string))


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('(?:\w{3,}|[\$\@()+.])+')
print(prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё. тт"))

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog = re.compile('(@\S+)')
string = prog.findall('abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')
for word in string:
  word = re.sub(',','',word)
  print(word)